In [6]:
import os
groq_api_key = os.environ.get("GROQ_API_KEY")
#print("groq_api_key =", groq_api_key)

In [7]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)


In [9]:
system_prompt="""
Translate the following input text to an RDF graph using the RDF, RDFS, and OWL ontologies formatted as TTL.
Use the prefix ex: with IRI <http://example.com/> for any created entities.
Use the prefix xsd: with IRI <http://www.w3.org/2001/XMLSchema#> for any data types.
Use the prefix owl: with IRI <http://www.w3.org/2002/07/owl#> for any owl: entities.
Use the prefix rdf: with IRI <http://www.w3.org/1999/02/22-rdf-syntax-ns#> for any rdf: entities.
Use the prefix rdfs: with IRI <http://www.w3.org/2000/01/rdf-schema#> for any rdfs: entities.

Include all necessary prefixes in the output.
Do not include any explanations or prose.
Do not include any text other than the generated RDF graph.
Do not include any additional information that is not explicitly requested.
Do not include any information that is not relevant to the input text.
Do not include any information that is not directly related to the input text.
Do not include any backtics ttl or turtle in the output.
"""

In [10]:
input_text="""
Elon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space
company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.
formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.
He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be
US$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended
the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through
his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.
Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics
and physics. He moved to California in 1995 to attend Stanford University, but dropped out after
two days and, with his brother Kimbal, co-founded online city guide software company Zip2.

"""

In [11]:
from langchain_core.messages import HumanMessage, SystemMessage
message1=[
    SystemMessage(content=system_prompt),
    HumanMessage(content=input_text)
]

In [12]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

In [15]:
chain=model|parser
output1=chain.invoke(message1)


In [16]:
from IPython.display import display, Markdown
display(Markdown(output1))

@prefix ex: <http://example.com/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:ElonMusk rdf:type ex:Person ;
    rdfs:label "Elon Reeve Musk" ;
    ex:birthDate "1971-06-28"^^xsd:date ;
    ex:knownFor ex:SpaceX , ex:Tesla ;
    ex:hasParent ex:MayeMusk ;
    ex:hasParent ex:ErrolMusk ;
    ex:attendedUniversity ex:UniversityOfPretoria ;
    ex:attendedUniversity ex:QueensUniversity ;
    ex:attendedUniversity ex:UniversityOfPennsylvania ;
    ex:attendedUniversity ex:StanfordUniversity ;
    ex:founded ex:TheBoringCompany , ex:xAI , ex:Neuralink , ex:OpenAI ;
    ex:owns ex:XCorp .

ex:Person rdfs:subClassOf owl:Thing .
ex:SpaceX rdf:type ex:Company ;
    rdfs:label "SpaceX" .
ex:Tesla rdf:type ex:Company ;
    rdfs:label "Tesla, Inc." .
ex:XCorp rdf:type ex:Company ;
    rdfs:label "X Corp." ;
    ex:wasFormerly ex:Twitter .
ex:TheBoringCompany rdf:type ex:Company ;
    rdfs:label "The Boring Company" .
ex:xAI rdf:type ex:Company ;
    rdfs:label "xAI" .
ex:Neuralink rdf:type ex:Company ;
    rdfs:label "Neuralink" .
ex:OpenAI rdf:type ex:Company ;
    rdfs:label "OpenAI" .
ex:Twitter rdf:type ex:Company ;
    rdfs:label "Twitter" .
ex:MayeMusk rdf:type ex:Person ;
    rdfs:label "Maye Musk" .
ex:ErrolMusk rdf:type ex:Person ;
    rdfs:label "Errol Musk" ;
    rdf:type ex:Engineer .
ex:KimbalMusk rdf:type ex:Person ;
    rdfs:label "Kimbal Musk" .
ex:UniversityOfPretoria rdf:type ex:University ;
    rdfs:label "University of Pretoria" .
ex:QueensUniversity rdf:type ex:University ;
    rdfs:label "Queen's University at Kingston" .
ex:UniversityOfPennsylvania rdf:type ex:University ;
    rdfs:label "University of Pennsylvania" .
ex:StanfordUniversity rdf:type ex:University ;
    rdfs:label "Stanford University" .
ex:Zip2 rdf:type ex:Company ;
    rdfs:label "Zip2" ;
    ex:foundedBy ex:ElonMusk , ex:KimbalMusk .





In [17]:
with open("Output1.ttl", "w") as ttl_file:
    ttl_file.write(output1)

In [8]:
from langchain_community.graphs import RdfGraph
graph = RdfGraph (
    source_file="Output1.ttl",
    standard="rdf",
)

In [9]:
graph.load_schema()
schema=graph.get_schema

In [10]:
sparql_generation_template="""
Write a SPARQL SELECT query for quering a rdf graph.
The query should retrieve the following information based on the given schema and question.
{schema}

Based on the above schema, write a SPARQL query that answers the following question:
{prompt}

Use only the classes and properties provided in the schema to construct the SPARQL query.
Do not use any classes and properties that are not explicitly provided in the SPARQL query.
Include all necessary prefixes.
Do not wrap the SPARQL query in triple backticks.
Do not include any text except the SPARQL query generated.

"""

In [11]:
from langchain.prompts import PromptTemplate
sparql_select_prompt=PromptTemplate(template=sparql_generation_template, input_variables=["schema","prompt"])

In [12]:
from langchain.prompts import PromptTemplate

qa_prompt_template="""
Generate a natural language response from the results of a SPARQL query.
Don't add any additional information.
Don't use internal knowledge to answer the question, just say you don't know if no information is available.
Information:
{context}

Question: {prompt}

Helpful Answer:
"""
qa_prompt=PromptTemplate(template=qa_prompt_template, input_variables=["context","prompt"])

In [22]:
from langchain.chains import GraphSparqlQAChain

chain = GraphSparqlQAChain.from_llm(
    model, 
    graph=graph, 
    verbose=False, 
    return_sparql_query=True, 
    qa_prompt=qa_prompt, 
    sparql_select_prompt=sparql_select_prompt,
    allow_dangerous_requests=True,
)

In [28]:
result = chain.invoke(
    input = {
        'schema': schema,
        'context': 'aswer from provided graph',
        'query': "What is Elon Musk's father name?"
    }
)

print(f"SQARQL query: {result['sparql_query']}")
print(f"Final answer: {result['result']}")

SQARQL query: PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ex: <http://example.com/>

SELECT ?fatherName
WHERE {
  ex:ElonMusk ex:hasParent ?father .
  ?father rdfs:label ?fatherName .
} 

Final answer: Errol Musk 

